1. Load the taxi environment and render it:

In [1]:
import gym
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



# Problem statement
The problem we are going to solve consists of a self-driving cab that has to pick up passengers and drop off to the right location, while doing as quick as possible and respecting the traffic rules.

There are 4 locations (R, G, B, Y) and the taxi hast to pick up the passenger at one location and drop him off at another.
- The agent receives +20 points for a successful dropoff
- It loses -1 point for every time-step. This way we encourage him to solve the enviroment as fast as possible: all the time it is on the road is consuming resources, such as fuel, so this negative reward may also have sense for this aspect.
- It is given a -10 points penalty for every illegal action it performs (in the pickup or dropoff actions).

# Solution
Reset the environment to a initial random state:

In [2]:
env.reset()
env.render()
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


## Action space
There are 6 possible actions that are encoded as follows:
- 0 = south
- 1 = north
- 2 = east
- 3 = west
- 4 = pickup
- 5 = dropoff

## State space
Taxi environment has 500 total possible states =
  - 5 rows ×
  - 5 cols ×
  - 5 (4 static +1 passenger inside taxi(4) locations) × [R(0), G(-), Y(-), B(2)]
  - 4 destinations

Show the value corresponding to an encoded state:

In [3]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

('State:', 328)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
# You can also manually set the state
env.s = 328
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Reward Table

It is defined as a Python dictionary  **P** that has as many entries  as possible actions (6). The default reward values are as follows:

In [5]:
env.P[state]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

The entries of the dictionary has this structure:

`{action: [(probability, nextstate, reward, done)]}`

- The key is the action number as encoded above
- We set their probabilities to 1.0. This way all actions are equally possible when the agent hast to take a random decission.
- All the movement actions provide a penalty of -1 to account for the time it runs before the passenger is drop off at the destination
- Both the pickup and drop off actions have -10 penalty
- If we are in a state where the taxi has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action (5)
- `done` signs a successful passenger drop off (in the right location). This event marks the succesful completion of an **episode**

## Solution using brute force
Iterate until one passenger reaches one destination (the agent completes one episode) = received reward is 20

In [71]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 4162
Penalties incurred: 1365


In [59]:
# This snippet replays all the steps of the brute force solution above 
from IPython.display import clear_output
from time import sleep

def print_frames(frames, delay = 0.1):
    for i, frame in enumerate(frames):
        if i==1:
            break
        clear_output(wait=True)
        print frame['frame']
        print "Timestep: {}".format(i + 1)
        print "State: {}".format(frame['state'])
        print "Action: {}".format(frame['action'])
        print "Reward: {}".format(frame['reward'])
        sleep(delay)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

Timestep: 1
State: 261
Action: 1
Reward: -1


The agent may take hundreds or even thousands of timesteps and makes lots of wrong drop offs in front of only one sucessful delivery.

## Using Reinforcement Learning
Q-learning algorithm lets the agent keep track of rewards to learn the best action for every single state.
- Each time an **action** is taken from a given **state**, a **reward** is obtained according to P
- The **reward** associated to each pair **(state, action)** creates a Q-table that is a 500 x 6 matrix 

The Q-value for a concrete pair state-action stand for the "quality" of that action in that state. Hence for a completely trained model, we will have a full 500 x 6 matrix, i.e., 3000 Q-values.
- Every row represents an state
- The maximum Q-value in such row let the agent know what action is the best to take in that state

Q-values are initialized to an arbitrary value, and as the agent exposes itself to the environment and receives different rewards by executing different actions, the Q-values are updated using the equation:

`Q(state,action) ← (1−α)Q(state,action) + α(reward + γ maxQ(next state,all actions))`
- **α** is the learning rate (0<α≤1)
- **γ** is the discount factor (0≤γ≤1) and determines how much importance we want to give to future rewards.
   -  If this factor is 0 makes the agent to consider only the immediate reward, making it to behave in a **greedy** manner

A trade-off arises between **exploration** (= taking a random action) and **exploitation** (using learned Q-values to decide the action).
We need to balance these two behaviors, and for that we introduce the parameter **ϵ "epsilon"** that represent the percentage of actions that should be of the  **exploration** type.
This prevents the agent to follow a single route (because it only knows a limited set of rewards, those of the states it visits = **overfitting**)

### Implementing Q-learning

Initialize Q-table to 0:

In [12]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

- Learning rate: `α = 0.1`
- Discount factor: `γ = 0.6` (don't be greedy, be patient and postpone the reward)
- Exploration rate: `ϵ = 0.1` (1-ϵ = 90% is explotation)

In [13]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    frames = [] # for animation
    
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    n_steps = 0 # Index for counting how many steps in each episode
    while not done:
        n_steps += 1
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action)
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        
        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 1000 == 0:
        sleep(1)
        print "Episode: {} Reward= {}, Number of steps= {}".format(i, reward, n_steps)
        clear_output(wait=True)
        
print("Training finished.\n")
#print frames

Training finished.

CPU times: user 26.9 s, sys: 79.7 ms, total: 27 s
Wall time: 2min 6s


Now that the Q-table has been established over 100,000 episodes, let's see what the Q-values are at our illustration's state:

In [14]:
q_table[328]

array([ -2.40437772,  -2.27325184,  -2.40857979,  -2.35569503,
       -10.7548625 , -10.86157945])

The max Q-value is "north" (-2.273) and this is the action to be taken from state 328

### Evaluating the agent

In [70]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    frames = [] # for animation
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1
        
        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            })
        
    total_penalties += penalties
    total_epochs += epochs

print_frames(frames, 1)
print "This was the replay of the LAST episode: \n"

print "Results after {} episodes:".format(episodes)
print " - Average timesteps per episode: {}".format(total_epochs / episodes)
print " - Average penalties per episode: {}".format(total_penalties / episodes)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

Timestep: 1
State: 367
Action: 1
Reward: -1
This was the replay of the LAST episode: 

Results after 100 episodes:
 - Average timesteps per episode: 12
 - Average penalties per episode: 0


## Hyperparameters
How to chose  the values of `alpha`, `gamma`, and `epsilon`? It will be mainly based on both intuition and hit and trial. In any case,  the three of them should decrease over time as the agent learns the best actions:
- decreasing the need of learning (`alpha`)
- As an optimum strategy is learned, there is no need to weight further actions in the next state. The agent knows well what will happen (`gamma`)
- When the environment is fully explored, random actions  have no sense (`epsilon`)